<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, Input
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [155]:
(ds_train, ds_test), ds_info = tfds.load('cifar100', split = ['train', 'test'], shuffle_files=True,
                             as_supervised = True, with_info = True,batch_size=16)

ds_train_subset = ds_train.take(1000)
# ds_train_subset = ds_train_subset.batch(16)
ds_val = ds_train.skip(1100)
# ds_val = ds_val.batch(16)

In [156]:
CLASS_NUM = ds_info.features["label"].num_classes

In [157]:
tf.keras.backend.clear_session()
model = Sequential([
    # Input(shape = (32, 32, 3)),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),

    Dense(CLASS_NUM,
          activation = 'softmax')

    ])
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [158]:
model.fit(ds_train_subset, validation_data = ds_val, epochs = 3)

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node Equal defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-package